In [12]:
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys
import time

import sknn
import theano
import theano.tensor as T
import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize

from functools import reduce

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from six.moves import cPickle as pickle

from sklearn.neural_network import MLPClassifier
from collections import Counter

%matplotlib inline

Import libraries

In [13]:
files = [
    'data_batch_1',
    'data_batch_2',
    'data_batch_3',
    'data_batch_4',
    'data_batch_5',
    'test_batch'
]
data = []
labels = []
start = time.time()
for file in files:
    with open(file, 'rb') as f:
        d = pickle.load(f, encoding='bytes')
        if file == 'test_batch':
            test_data = d[b'data']
            test_labels = d[b'labels']
        else:
            data.append(d[b'data'])
            labels.append(d[b'labels'])
end = time.time()
print('Time to load data: {:.3f}s'.format(end - start))
for i in range(len(data)):
    print('Train data {}:'.format(i), data[i].shape, len(labels[i]))
print('Test data:', test_data.shape, len(test_labels))

merged_data = reduce(lambda a,b: np.vstack((a,b)), data)
merged_labels = reduce(lambda a,b: a+b, labels)
print('Merged train data:', merged_data.shape, len(merged_labels))

Time to load data: 0.157s
Train data 0: (10000, 3072) 10000
Train data 1: (10000, 3072) 10000
Train data 2: (10000, 3072) 10000
Train data 3: (10000, 3072) 10000
Train data 4: (10000, 3072) 10000
Test data: (10000, 3072) 10000
Merged train data: (50000, 3072) 50000


Load data and print out time- this code is taken directly from section

In [14]:
scaler = StandardScaler()
scaler.fit(merged_data) 
#Train our scaler based on our training data

train_data = merged_data[:]
train_labels = merged_labels[:]
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
#Apply our scaling to our training and test data, creating a copy of our merged data called 'train_data'

C:\Users\Shadi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Scaling all the data so the Neural Network will work as expected

In [15]:
mlp = MLPClassifier(solver='lbgfs', alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)
start = time.time()
mlp.fit(train_data, train_labels)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Create and train a basic Multi Layer Perceptron model

Time to build: 171.217s


In [16]:
predictions = mlp.predict(train_data) #Store the predictions for this basic model
correct = merged_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
accuracy = (correct == 0).sum() / len(correct)
accuracy * 100
#This is the accuracy of the base model on the training data

32.013999999999996

In [17]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

30.34

Our basic MLP model seems to have an accuracy of roughly 30-32% to start- looks like there's a lot of optimization we can do on this

http://lasagne.readthedocs.io/en/latest/user/tutorial.html

The documentation for the Lasagne package was extensively utilized for creating the following Neural Network.

In [18]:
x_train = merged_data[:]
x_train = x_train.reshape(50000, 3, 32, 32)
y_train = train_labels[:]
#Prepare data for lasagne

In [19]:
def build_mlp(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                     input_var=input_var)
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)
    l_hid1 = lasagne.layers.DenseLayer(
        l_in_drop, num_units=800,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    l_hid2 = lasagne.layers.DenseLayer(
        l_hid1_drop, num_units=800,
        nonlinearity=lasagne.nonlinearities.rectify)

    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)
    
    l_out = lasagne.layers.DenseLayer(
        l_hid2_drop, num_units=10,
        nonlinearity=lasagne.nonlinearities.softmax)
    return l_out

In [20]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
network = build_mlp(input_var)
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [24]:
train_fn = theano.function([x_train, y_train], loss)

TypeError: ("Unknown input type: <class 'numpy.ndarray'>, expected Variable instance", array([[[[ 59,  43,  50, ..., 158, 152, 148],
         [ 16,   0,  18, ..., 123, 119, 122],
         [ 25,  16,  49, ..., 118, 120, 109],
         ..., 
         [208, 201, 198, ..., 160,  56,  53],
         [180, 173, 186, ..., 184,  97,  83],
         [177, 168, 179, ..., 216, 151, 123]],

        [[ 62,  46,  48, ..., 132, 125, 124],
         [ 20,   0,   8, ...,  88,  83,  87],
         [ 24,   7,  27, ...,  84,  84,  73],
         ..., 
         [170, 153, 161, ..., 133,  31,  34],
         [139, 123, 144, ..., 148,  62,  53],
         [144, 129, 142, ..., 184, 118,  92]],

        [[ 63,  45,  43, ..., 108, 102, 103],
         [ 20,   0,   0, ...,  55,  50,  57],
         [ 21,   0,   8, ...,  50,  50,  42],
         ..., 
         [ 96,  34,  26, ...,  70,   7,  20],
         [ 96,  42,  30, ...,  94,  34,  34],
         [116,  94,  87, ..., 140,  84,  72]]],


       [[[154, 126, 105, ...,  9